In [1]:
%reload_ext autoreload
%autoreload 2

import h5py
N_Cs = 8

with h5py.File('../dxtb/dxtb-gpu/gpu-cpu_analysis/rdkit/alkanes_data_500.hdf5', 'r') as f:
    for mol_name, data in f.items():
        if mol_name == f"alkane_{N_Cs}_carbons":
            atomic_numbers = data['atomic_numbers'][:]
            coordinates = data['coordinates'][:]

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")

Number of carbon atoms in alkane_9_carbons: 8
Nb of atoms: 26


# Batched

In [2]:
import dxtb
from dxtb._src.typing import DD
import torch
from ase.build import molecule

opts = {"scf_mode": "implicit", "batch_mode": 1, "int_driver": "libcint"}
batch_size = 64
results = {}

for device in ["cuda:0", "cpu"]:
    print(f"\nDevice: {device}")
    dd = {"dtype": torch.float32, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
    positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
    numbers = torch.stack([numbers] * batch_size)
    positions = torch.stack([positions] * batch_size).requires_grad_()
    charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
    
    dxtb.timer.reset()
    e = calc.get_energy(positions, chrg=charges)
    dxtb.timer.start("Forces autograd")
    forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
    dxtb.timer.stop("Forces autograd")
    dxtb.timer.print(v=0)

    results[device] = {
        "energy": e.detach().cpu(),
        "forces": forces.detach().cpu()
    }

# Compare results
energy_diff = (results["cuda:0"]["energy"] - results["cpu"]["energy"]).abs().max()
forces_diff = (results["cuda:0"]["forces"] - results["cpu"]["forces"]).abs().max()

print(f"\n[Comparison]")
print(f"GPU energy: {results['cuda:0']['energy'].mean().item():.6e}")
print(f"CPU energy: {results['cpu']['energy'].mean().item():.6e}")
print(f"Max energy diff: {energy_diff.item():.6e}")
print(f"Max forces diff: {forces_diff.item():.6e}")


Device: cuda:0


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.046           5.09
 - Repulsion                0.028          61.06
 - DispersionD3             0.013          27.54
 - Halogen                  0.005          11.11
Integrals                   0.332          36.98
 - Overlap                  0.328          99.01
 - Core Hamiltonian         0.003           0.98
SCF                         0.299          33.31
 - Interaction Cache        0.001           0.31
 - Potential                0.198          66.20
 - Fock build               0.001           0.46
 - Diagonalize              0.147          49.12
 - Density                  0.007           2.23
 - Charges                  0.007           2.26
get_refocc                  0.016           1.74
get_guess                   0.018           2.05
Chrg init                   0.000           0.00
guess init scp              0.001 

In [ ]:
import torch
import dxtb
from dxtb._src.typing import DD
import ase.build.molecule as molecule
import torch.profiler

opts = {"scf_mode": "implicit", "batch_mode": 2, "int_driver": "libcint"}
batch_size = 100

for device in ["cpu", "cuda:0"]:
    dd = {"dtype": torch.float64, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
    positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
    numbers = torch.stack([numbers] * batch_size)
    positions = torch.stack([positions] * batch_size).requires_grad_()
    charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

    torch.cuda.synchronize() if "cuda" in device else None
    dxtb.timer.reset()
    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)

    with torch.profiler.profile(
        activities=[
            torch.profiler.ProfilerActivity.CPU,
            torch.profiler.ProfilerActivity.CUDA
        ] if "cuda" in device else [torch.profiler.ProfilerActivity.CPU],
        record_shapes=True,
        profile_memory=True,
        with_stack=True
    ) as prof:
        e = calc.get_energy(positions, chrg=charges)
        dxtb.timer.start("Forces autograd")
        forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
        dxtb.timer.stop("Forces autograd")
        torch.cuda.synchronize() if "cuda" in device else None

    dxtb.timer.print(v=0)
    print(f"\n--- PyTorch Profiler Summary ({device}) ---")
    print(prof.key_averages().table(sort_by="cuda_time_total" if "cuda" in device else "cpu_time_total", row_limit=20))


# Non-Batched

In [ ]:
import dxtb
from dxtb._src.typing import DD
import torch
import ase.build.molecule as molecule

opts = {"scf_mode": "implicit", "batch_mode": 0}

for device in ["cpu", "cuda:0"]:
    dd = {"dtype": torch.float64, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device= dd["device"], dtype = torch.int64)
    positions = torch.tensor(coordinates, device = dd['device'], dtype = torch.float64).requires_grad_()

    dxtb.timer.reset()
    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts)
    e = calc.get_energy(positions)
    forces = calc.get_forces(positions)
    dxtb.timer.print(v=0)


# Line plot

In [ ]:
import dxtb
from dxtb._src.typing import DD
import torch
import ase.build.molecule as molecule
import matplotlib.pyplot as plt

# Setup
opts = {"scf_mode": "implicit", "batch_mode": 1}

batch_sizes = [2, 4, 8, 16, 32, 64, 128, 256]
timings = {"cpu": [], "cuda:0": []}

for batch_size in batch_sizes:
    for device in ["cpu", "cuda:0"]:
        dd: DD = {"dtype": torch.float64, "device": torch.device(device)}
        numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
        positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])

        numbers = torch.stack([numbers] * batch_size)
        positions = torch.stack([positions] * batch_size)
        charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

        dxtb.timer.reset()
        calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts)
        e = calc.get_energy(positions, chrg=charges)
        dxtb.timer.print(v=5)
        total = dxtb.timer.timers["total"].elapsed_time
        timings[device].append(total)
        print(f"[{device}] Batch {batch_size} → {total:.2f}s")

In [ ]:
plt.figure()

for device in timings:
    actual_times = timings[device]
    
    # Plot actual timings
    plt.plot(batch_sizes, actual_times, label=f"{device}", marker='o', linestyle='-')
    
    # Get anchor time at batch size 2
    anchor_batch_size = 4
    anchor_idx = batch_sizes.index(anchor_batch_size)
    anchor_time = actual_times[anchor_idx]
    
    # Generate linear scaling line
    linear_times = [anchor_time * (b / anchor_batch_size) for b in batch_sizes]
    plt.plot(batch_sizes, linear_times, linestyle='--', label=f"{device} (linear)", alpha=0.6)

plt.xlabel("Batch Size")
plt.ylabel("Total Time (s)")
plt.title("dxtb Total Time vs. Batch Size with Linear Scaling Baseline")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
